In [2]:
import os
import sys
sys.path.append('../src/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import cca_functions

# configurations
replace all values after the commented lines with the according values here

In [3]:
# the path to the data folder containing the MIA_ folders
data_dir = os.path.join('..', 'data', 'nd2_files')
# the MIA_ folder which should be analyzed
mia_dir = 'MIA_F016.nd2'
# the positions which should be incorporated in the analysis
positions = [3,4]
# channels to incorporate in the Analysis, assuming to be the same for all positions
channels = ['phase_contr', 'mCitrine']

# load data and perform all needed calculations on image data
(no need to keep all image data in memory at once, load it, calculate stuff, drop it)

In [4]:
# build some parameters based on configurations for further use
pos_strings = [f'Position_{str(pos)}' for pos in positions]
no_of_channels = len(channels)
file_dirs = [os.path.join(data_dir, mia_dir, pos_str, 'Images') for pos_str in pos_strings]

In [5]:
overall_df = pd.DataFrame()
for pos_idx, file_dir in enumerate(file_dirs):
    channel_data = ('placeholder')*no_of_channels
    print(f'Load files for position {positions[pos_idx]}...')
    *channel_data, seg_mask, cc_data = cca_functions.load_files(file_dir, channels)
    cc_data = cca_functions.rename_columns(cc_data)
    max_frame = cc_data.frame_i.max()
    print(f'Calculate regionprops for cells based on Segmentation...')
    rp_df = cca_functions.calculate_rp_df(seg_mask[:max_frame+1])
    print(f'Calculate sum of signals for every channel and cell...')
    flu_signal_df = cca_functions.calculate_flu_signal(seg_mask, channel_data, channels, cc_data)
    temp_df = cc_data.merge(rp_df, on=['frame_i', 'Cell_ID'], how='left')
    temp_df = temp_df.merge(flu_signal_df, on=['frame_i', 'Cell_ID'], how='left')
    temp_df['position'] = positions[pos_idx]
    temp_df['directory'] = file_dir
    overall_df = overall_df.append(temp_df).reset_index(drop=True)

Load files for position 3...
Calculate regionprops for cells based on Segmentation...
Calculate sum of signals for every channel and cell...
Load files for position 4...
Calculate regionprops for cells based on Segmentation...
Calculate sum of signals for every channel and cell...


# generate plots / analyses on obtained data

In [6]:
overall_df

,frame_i,Cell_ID,cell_cycle_stage,generation_num,relative_ID,relationship,emerg_frame_i,division_frame_i,is_cell_excluded,area,...,lifespan,age,frames_till_gone,elongation,phase_contr_signal,phase_contr_signal_scaled,mCitrine_signal,mCitrine_signal_scaled,position,directory
0,0,1,S,0,2,bud,-1,-1,False,1204.0,...,122.0,1.0,121.0,1.055982,2840728.0,127.987599,1428190.0,146.012374,3,..\data\nd2_files\MIA_F016.nd2\Position_3\Images
1,0,2,S,1,1,mother,-1,-1,False,2563.0,...,122.0,1.0,121.0,1.341889,6631870.0,314.608147,2481926.0,236.632142,3,..\data\nd2_files\MIA_F016.nd2\Position_3\Images
2,1,1,S,0,2,bud,-1,-1,False,1413.0,...,122.0,2.0,120.0,1.091615,3550165.0,165.801009,1545063.0,154.649557,3,..\data\nd2_files\MIA_F016.nd2\Position_3\Images
3,1,2,S,1,1,mother,-1,-1,False,2590.0,...,122.0,2.0,120.0,1.328818,6513439.0,304.346720,2509621.0,239.361595,3,..\data\nd2_files\MIA_F016.nd2\Position_3\Images
4,2,1,S,0,2,bud,-1,-1,False,1492.0,...,122.0,3.0,119.0,1.076634,3670872.0,169.463014,1598222.0,156.844638,3,..\data\nd2_files\MIA_F016.nd2\Position_3\Images
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2329,122,26,S,0,1,bud,112,-1,False,1438.0,...,11.0,11.0,0.0,1.119942,3058857.0,116.525934,1747603.0,161.308605,4,..\data\nd2_files\MIA_F016.nd2\Position_4\Images
2330,122,27,S,0,4,bud,113,-1,False,1282.0,...,10.0,10.0,0.0,1.150463,2874701.0,113.897688,1355723.0,117.153116,4,..\data\nd2_files\MIA_F016.nd2\Position_4\Images
2331,122,28,S,0,14,bud,115,-1,False,1737.0,...,8.0,8.0,0.0,1.107677,3615414.0,135.367144,1870341.0,163.140466,4,..\data\nd2_files\MIA_F016.nd2\Position_4\Images
2332,122,29,S,0,13,bud,117,-1,False,962.0,...,6.0,6.0,0.0,1.192430,2357746.0,99.068547,1372806.0,134.752668,4,..\data\nd2_files\MIA_F016.nd2\Position_4\Images
